In [15]:
import numpy as np
import scipy as sp

# Helper Functions

In [16]:
def get_classes(labels):
    # Post data cleaning 
    return np.unique(labels).astype(int)

In [17]:
def euc(vector, matrix):
    d = np.sum((vector - matrix)**2, axis = 1, keepdims = True)
    return np.sqrt(d.T)

In [18]:
def pairwise_distance(test_data, train_data):
    d = np.zeros((test_data.shape[1], train_data.shape[1]))
    for i,v in enumerate(test_data.T):
        d[i,:] = euc(v,train_data.T)
    return d

In [19]:
def infer_class (closeness, data_labels, classes):
    
    # first, try to do a simple majority vote
    #res_class = sp.stats.mode(class_labels, axis = 0)
    hist = np.histogram(data_labels, bins = len(classes), range = (min(classes), 
max(classes)))
 
    # check if we have a clear majority
    s = np.flip(np.sort(hist[0]))
    # if not, do a histgram weighted with the closeness
    if s[0] == s[1]:
        hist = np.histogram(data_labels, bins = len(classes), range = 
(min(classes), max(classes)), weights = closeness)
 
    return classes[np.argmax(hist[0])]

In [33]:
def comp_confmat(gt, est, class_indices):
    conf_mat = np.zeros((len(class_indices), len(class_indices)))
    for i,row in enumerate(gt):
        conf_mat[row-class_indices[0],est[i]-class_indices[0]] += 1
    return conf_mat.astype(int)

In [36]:
def eval(gt, est, class_indices):
    # compute confusion matrix
    conf_mat = comp_confmat(gt, est, class_indices)
    accuracy = np.trace(conf_mat)/len(gt)
    return (accuracy, conf_mat)

In [37]:
def find_best_features_with_accuracy(data, labels, k, num_folds, sel_features = 
[]):
    accuracy = np.zeros(data.shape[0])
    for f,feature_value in enumerate(data):
        if len(sel_features) > 0:
            if np.where(sel_features == f)[0].size > 0: # replace with np.isin
                continue
        [accuracy[f], dummy, dummy] = cross_validate(np.vstack((feature_value, 
data[sel_features,:])), labels, k, num_folds)
    return (np.argmax(accuracy).astype(int), np.max(accuracy))

In [42]:
def split_data (gt_labels, num_folds, classes):
    num_obs = np.zeros(len(classes))
    for k,c in enumerate(classes):
        num_obs[k] = int(len(gt_labels[gt_labels == c]))
    num_obs = num_obs.astype(int)
    last = np.zeros(len(classes)).astype(int)
    data_ind = [ [] for _ in range(num_folds)] 
    for f in range(num_folds):
        for k,c in enumerate(classes):
            num = int(np.floor(num_obs[k]/num_folds))
            data_ind[f] = data_ind[f] + np.argwhere(gt_labels == c)[np.arange(last[k],last[k]+num)].astype(int).tolist()
            last[k] +=num
    for k,c in enumerate(classes):
        if num_obs[k]-last[k] > 0:
            data_ind[num_folds-1] = data_ind[num_folds-1] + np.argwhere(gt_labels 
== c)[last[k]:num_obs[k]].astype(int).tolist()
    for f in range(num_folds):
        data_ind[f] = np.ravel(data_ind[f])
    return data_ind

In [20]:
def knearestneighbor(test_data, train_data, train_label, k=1):
    num_features = test_data.shape[0]
    classes = np.unique(train_label).astype(int)
    est_class = -1*np.ones(test_data.shape[1])
    d = pairwise_distance(test_data, train_data)
    ind = np.argsort(d, axis = 1).astype(int)
    ind = ind[:,range(k)]
    ma = np.amax(d)
    if ma <= 0:
        ma = 1
    d = 1-d/ma
    for obs,index in enumerate(ind):
        est_class[obs] = infer_class (d[obs,index], train_label[index], classes)
    return np.squeeze(est_class.astype(int))

In [38]:
def cross_validate(data, labels, k, num_folds):
    # init result
    classes = get_classes(labels)
    fold_accuracies = np.zeros(num_folds)
    conf_mat = np.zeros((num_folds, len(classes), len(classes))) 
    # split data
    fold_ind = split_data(labels, num_folds, classes)
    # do classification for each fold
    for n in range(num_folds):
        train_data = np.zeros((data.shape[0],0))
        train_label = np.zeros(0)
        #split train and testdata
        for i,train in enumerate(fold_ind):
            if i == n:
                test_data = data[:,np.squeeze(fold_ind[n])]
                test_label = labels[np.squeeze(fold_ind[i])].astype(int)
                continue
            train_data = np.hstack((train_data, data[:, fold_ind[i]]))
            train_label = np.append(train_label, labels[fold_ind[i]])
        # classify
        est_label = knearestneighbor(test_data, train_data, train_label, k)
        # evaluate result
        [fold_accuracies[n], conf_mat[n,:,:]] = eval(test_label, est_label, 
classes)
    
    avg_accuracy = np.mean(fold_accuracies)
    conf_mat = np.sum(conf_mat,axis=0)
    return (avg_accuracy, fold_accuracies, conf_mat)
    

In [39]:
def select_features(data, labels, k, num_folds):
    sel_feature_ind = -1*np.ones(data.shape[0]).astype(int)
    accuracy =  np.zeros(data.shape[0])
    for f in range(data.shape[0]):
        [sel_feature_ind[f], accuracy[f]] = find_best_features_with_accuracy(data, 
labels, k, num_folds, sel_feature_ind[range(f)])
    return (sel_feature_ind, accuracy)


In [41]:
def evaluate(data, labels, num_folds = 10, k = np.array([1,3,7]), sel_features = 
np.array([0,1,2,3,4,5,6,7,9])):
    
    # init result
    classes = get_classes(labels)
    accuracies = np.zeros(len(k)) 
    conf_matrices = np.zeros((len(k), len(classes), len(classes)))
    # run the cross val with the different k values
    for n in range(len(k)):
        [accuracies[n], _, conf_matrices[n]] = cross_validate(data[sel_features,:],
labels, k[n], num_folds)
    return (accuracies, conf_matrices)

In [43]:
def find_best_features(data, labels, k, num_folds):
    [index, acc] = find_best_features_with_accuracy(data, labels, k, num_folds)
    return index

In [45]:
labels = np.loadtxt('/Users/noelalben/Downloads/data/labels.txt')

In [48]:

classes = np.unique(labels)

In [50]:
for c in classes:
    print(len(labels[labels==c]))

100
100
100
100
100
